# Part 1: Decomposing Representations into Irreps

**6.7970/8.750 Symmetry and its Application to Machine Learning**

We work through the decomposition algorithm step by step using $D_4$ (the symmetry group of a square, order 8). By the end you will:

1. Build the **regular representation** and understand why it contains all irreps
2. Walk through the **decomposition algorithm** (commutant → random combination → eigenspaces)
3. Find and **label all 5 irreps** of $D_4$
4. Build the **character table** and match it to the standard convention

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/atomicarchitects/symm4ml-colabs/blob/main/decomposition_lecture/01_decomposing_reps.ipynb)

## Setup

In [ ]:
%%capture
!pip install https://symm4ml.mit.edu/_static/symm4ml_s26/symm4ml/symm4ml_latest.zip

In [ ]:
import numpy as np
from symm4ml import groups, linalg, rep, vis
from symm4ml.utils import match_character_tables, print_character_table


---
## 1. Building the $D_4$ group

$D_4$ is the symmetry group of a square: 4 rotations and 4 mirrors, so $h = |D_4| = 8$.

We can generate the full group from just **two generators**:
- A **4-fold rotation** $C_4$ (by $90°$)
- A **mirror** $\sigma_v$ (across the $x$-axis)

In [ ]:
# Two generators for D4
C4 = np.array([[np.cos(np.pi/2), -np.sin(np.pi/2)],
               [np.sin(np.pi/2),  np.cos(np.pi/2)]])  # 90° rotation

sigma_v = np.array([[-1., 0.],
                    [ 0., 1.]])  # mirror across x-axis

# Generate the full group by taking all products
D4 = groups.generate_group(np.array([sigma_v, C4]))

# Put identity first for convenience
D4 = np.array(D4[::-1])

table_D4 = groups.make_multiplication_table(D4)
h = len(table_D4)
print(f"D4 has {h} elements (2D matrices of shape {D4[0].shape})")
print(f"Conjugacy classes: {len(groups.conjugacy_classes(table_D4))}")

---
## 2. The regular representation

The **regular representation** is built directly from the multiplication table:
$D^{\text{reg}}(g)|f\rangle = |gf\rangle$. Each matrix is $h \times h$.

**Why it matters**: The regular representation contains **every** irrep of the group, each appearing $\ell_j$ times (where $\ell_j$ is the dimension of irrep $\Gamma_j$). So by decomposing it, we discover all irreps.

In [ ]:
D4_reg = rep.regular_representation(table_D4)
print(f"Regular rep shape: {D4_reg.shape}  (8 matrices, each 8×8)")
print(f"\nD_reg(E) is the 8×8 identity (as expected):")
print(D4_reg[0].astype(int))

In [ ]:
# Visualize all 8 matrices of the regular representation
vis.plot_matrices(D4_reg, figsize=(12, 3));

Each matrix is a permutation matrix — it shuffles the 8 basis vectors according to the group multiplication.

---
## 3. The decomposition algorithm, step by step

The goal: find a change of basis that block-diagonalizes the regular representation.

### Step 1: Find the commutant

We find all matrices $Q$ that commute with the representation: $Q \, D^{\text{reg}}(g) = D^{\text{reg}}(g) \, Q$ for all $g$.

By **Schur's Lemma**, the number of independent solutions tells us about the representation's structure.

In [ ]:
Qs = linalg.infer_change_of_basis(D4_reg, D4_reg)
print(f"Number of independent Q matrices: {len(Qs)}")
print(f"\nFor an irrep we'd get 1 (just c·I).")
print(f"Getting {len(Qs)} confirms the regular rep is highly reducible!")

### Step 2: Form a random linear combination

Any linear combination $Q' = \sum_i \alpha_i Q_i$ also commutes with the rep.
With random $\alpha_i$, distinct irrep blocks will (with probability 1) get **distinct eigenvalues**.

In [ ]:
np.random.seed(42)
alpha = np.random.randn(len(Qs))
Q_rand = np.einsum('n,nij->ij', alpha, Qs)

print(f"Q_rand shape: {Q_rand.shape}")
vis.plot_matrices(Q_rand[np.newaxis], figsize=(3, 3));

### Step 3: Eigenspaces reveal the irreps

Each **eigenspace** of $Q'$ corresponds to one irrep (or multiple copies of the same irrep if they share an eigenvalue — but random $\alpha$ prevents this).

In [ ]:
val, vec = np.linalg.eig(Q_rand)
spaces = linalg.eigenspaces(val, vec)

print(f"Found {len(spaces)} eigenspaces:\n")
for eigenvalue, eigenvectors in spaces:
    dim = eigenvectors.shape[1]
    print(f"  eigenvalue = {eigenvalue:7.3f}  →  eigenspace dimension = {dim}")

dims = [ev.shape[1] for _, ev in spaces]
print(f"\nDimensions: {dims}")
print(f"Sum of dimensions: {sum(dims)} = h = {h}  ✓")

We get 6 eigenspaces: four of dimension 1 and **two** of dimension 2.

The four dim-1 eigenspaces correspond to the four 1D irreps ($A_1, A_2, B_1, B_2$).

The two dim-2 eigenspaces each give one copy of the 2D irrep $E$. In the regular representation, $E$ appears $\ell_E = 2$ times. The random $Q'$ acts on the two copies as $M \otimes I_2$ (a 2×2 matrix on the multiplicity space, tensored with identity on the irrep space). Each eigenvalue of $M$ gives a dim-2 eigenspace — one particular linear combination of the two copies. We could mix these two copies any way we like and still get valid copies of $E$.

---
## 4. Finding all irreps with `infer_irreps`

The function `rep.infer_irreps` wraps the algorithm above: it decomposes the regular representation and returns the **unique** irreps.

In [ ]:
np.random.seed(42)
D4_irreps = rep.infer_irreps(table_D4)

print(f"Found {len(D4_irreps)} unique irreps:")
for i, ir in enumerate(D4_irreps):
    print(f"  Irrep {i}: dim {ir.shape[1]}")

### A note on ordering

`infer_irreps` returns irreps in an **arbitrary order** that depends on the random seed. The standard convention for $D_4$ labels them $A_1, A_2, B_1, B_2, E$. We need to figure out which is which.

---
## 5. Labeling the irreps

To match our irreps to the standard labels, we:
1. Compute the **character table** (traces of each irrep at each conjugacy class)
2. Compare to the [reference character table for $D_4$](http://gernot-katzers-spice-pages.com/character_tables/D4.html)

First, let's get the conjugacy classes and compute characters.

In [ ]:
conj_classes = groups.conjugacy_classes(table_D4)

print(f"{len(conj_classes)} conjugacy classes (= {len(D4_irreps)} irreps, as expected):\n")
for cc in conj_classes:
    print(f"  {set(cc)}  (size {len(cc)})")

In [ ]:
# Compute the character table (rows = irreps, cols = classes)
# in whatever order infer_irreps gave us
char_table_raw = rep.character_table(D4_irreps, conj_classes)

print("Character table (unordered):\n")
print(np.round(char_table_raw.real, 2))

### Matching to the standard convention

The reference character table for $D_4$ is:

| | $E$ | $2C_4$ | $C_2$ | $2\sigma_v$ | $2\sigma_d$ |
|---|---|---|---|---|---|
| $A_1$ | 1 | 1 | 1 | 1 | 1 |
| $A_2$ | 1 | 1 | 1 | −1 | −1 |
| $B_1$ | 1 | −1 | 1 | 1 | −1 |
| $B_2$ | 1 | −1 | 1 | −1 | 1 |
| $E$ | 2 | 0 | −2 | 0 | 0 |

We can identify each irrep by its **character signature** (the set of character values). Let's reorder both the irreps and the conjugacy classes to match.

In [ ]:
# Reference character table for D4 (standard convention)
# Columns: E, 2C4, C2, 2σv, 2σd
names = ['A₁', 'A₂', 'B₁', 'B₂', 'E']
class_names = ['E', '2C₄', 'C₂', '2σᵥ', '2σ_d']
ref_chars = np.array([
    [1,  1,  1,  1,  1],   # A1
    [1,  1,  1, -1, -1],   # A2
    [1, -1,  1,  1, -1],   # B1
    [1, -1,  1, -1,  1],   # B2
    [2,  0, -2,  0,  0],   # E
])

# Compute character table in whatever order we currently have
conj_list = list(conj_classes)  # fix the iteration order
char_table = rep.character_table(D4_irreps, conj_list)

# Find the row (irrep) and column (class) permutations
row_perm, col_perm = match_character_tables(char_table, ref_chars)

print("Matching our irreps to standard D₄ labels:")
for std_idx, our_idx in enumerate(row_perm):
    print(f"  D4_irreps[{our_idx}] → {names[std_idx]}")

In [ ]:
# Apply the reordering
D4_irreps_std = [D4_irreps[i] for i in row_perm]
conj_std = [conj_list[j] for j in col_perm]

# Print the character table in standard order
char_table_std = rep.character_table(D4_irreps_std, conj_std)
print_character_table(char_table_std, names, class_names,
                      title="Character table of D₄ (standard ordering)")


### Visualizing the irreps

Now let's see the actual matrices for each irrep, in standard order.

In [ ]:
for name, ir in zip(names, D4_irreps_std):
    print(f"\n{name} (dim {ir.shape[1]}):")
    vis.plot_matrices(ir, figsize=(8, 2));

**Observations**:
- The four 1D irreps ($A_1, A_2, B_1, B_2$) are just $\pm 1$ scalars.
- The 2D irrep $E$ contains the original 2D rotation/mirror matrices (up to a similarity transform).

We can verify this: the 2D irrep $E$ should be isomorphic to our original $D_4$ matrices.

In [ ]:
E_irrep = D4_irreps_std[-1]  # the 2D irrep
print(f"Is the 2D irrep isomorphic to our original D4 matrices?")
print(f"  {rep.are_isomorphic(E_irrep, D4)}")

# Find the change of basis
Q = linalg.infer_change_of_basis(E_irrep, D4)
print(f"\nChange of basis (rotation relating them):")
print(np.round(Q[0], 4))

### Direct sum: one copy of each irrep

Stacking all irreps into a direct sum gives a block-diagonal representation of dimension $1+1+1+1+2 = 6$. This is **not** the regular rep (which has dimension 8) — each irrep appears only once here, whereas in the regular rep each appears $\ell_j$ times. The block-diagonal structure is clearly visible!

In [ ]:
direct_all = rep.direct_sum_multiple(D4_irreps_std)
print(f"Direct sum shape: {direct_all.shape} (dim = {direct_all.shape[1]})")
vis.plot_matrices(direct_all, figsize=(10, 3));